In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# 'target'을 y_train에 할당하고, 나머지 특성들을 X_train에 할당합니다.
columns_to_drop = ['target']
if 'pred' in dt_train.columns:
    columns_to_drop.append('pred')
if 'error' in dt_train.columns:
    columns_to_drop.append('error')

y_train = dt_train['target']
X_train = dt_train.drop(columns=columns_to_drop, axis=1)

# Hold-out Split: 학습 데이터와 검증 데이터를 8:2로 나누기
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

# XGBoost 모델 정의
model = xgb.XGBRegressor(
    n_estimators=1000,            # 트리의 개수
    max_depth=10,                  # 트리의 최대 깊이
    learning_rate=0.05,           # 학습률
     # 회귀 문제에서 사용되는 손실 함수
    random_state=1,              # 랜덤 시드 설정
                       # 모든 CPU 코어를 사용하여 병렬 처리
)             

# 하이퍼파라미터 튜닝을 위한 그리드 서치
param_grid = {
    'n_estimators': [9000, 10000, 11000, 12000],
    'max_depth': [5, 6, 7],
    'learning_rate': [0.05, 0.075, 0.1],
    'min_child_weight': [1, 3]
}

# GridSearchCV를 사용해 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# 학습
grid_search.fit(X_train, y_train)

# 최고의 모델 출력
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# 예측 수행
pred = best_model.predict(X_val)

# Mean Squared Error (MSE)로 모델 성능 평가
mse = mean_squared_error(y_val, pred)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")

# 예측값 출력 (선택 사항)
print("Predictions:", pred)

# Feature Importance
import matplotlib.pyplot as plt
import xgboost as xgb

# 특성 중요도 시각화
xgb.plot_importance(best_model)
plt.show()
